### Dico to liste

In [39]:
# dictionnaire mot fréquence : dico
dico = {'il':5 , 'est':2 , 'voiture':3, 'un': 7}

In [5]:
def simplification(dico): 
    vocab=[]
    freq=[]
    for key, value in dico.items():
        vocab+=[key]
        freq+=[value]
    return vocab, freq

In [40]:
vocab, freq=simplification(dico)
vocab

['il', 'est', 'voiture', 'un']

### Synchronisation indice - id

In [ ]:
# Créer liste pour chopper le texte (l'indice de cette liste synchronise avec l'id des pages)

In [7]:
import xml.etree.ElementTree as ET
tree = ET.parse("qlqpage.xml")
root = tree.getroot()

In [9]:
for page in root:
    for text in page.findall('text'):
        contenu = text.text
        print("lol")

lol
lol
lol
lol
lol


### Liens internes

In [25]:
def liens_internes(text): 
    liens=[]
    for i in range(len(text)-1):
        if text[i]=='[' and text[i+1]=='[':
            debut=i+2
        if text[i]==']' and text[i+1]==']':
            fin = i-1
            ref = text[debut:fin+1]
            if not ref in liens:
                liens+=[ref]
            
    return liens

In [41]:
liens_internes(root[1].findall('text')[0].text)

['Organigramme de programmation',
 "algorithme d'Euclide",
 "algorithme d'Euclide]].",
 'algorithme',
 'problème algorithmique',
 'mathématicien',
 'Al-Khwârizmî',
 'Philippe Flajolet',
 'Moyen Âge',
 'Abrégé du calcul par la restauration et la comparaison|le premier ouvrage systématique',
 'équation linéaire|équations linéaires',
 'équation du second degré|quadratiques',
 'Josef Hoëné-Wronski|Wronski',
 'Josef Hoëné-Wronski|Hoéné de Wronski',
 'Université du Québec à Montréal|UQAM',
 "File:Cuneiform tablet- fragment of a mathematical problem text MET ME86 11 404.jpg|thumb|Fragment d'une tablette cunéiforme avec un problème algorithmique. MET ME86 11 404",
 'Babylone|Babyloniens',
 'Calcul (mathématiques)|calcul',
 'Équation|équations',
 'Donald Knuth',
 'Communications of the ACM',
 'Addison-Wesley',
 'Librairie Eyrolles',
 "algorithme d'Euclide|Éléments d'Euclide",
 'Plus grand commun diviseur|PGCD',
 'itération',
 "correction d'un algorithme|correction",
 'Archimède',
 'Pi|π',
 'Enc

### Construction dictionnaire Mot-page

In [43]:
pip install Unidecode

Note: you may need to restart the kernel to use updated packages.


In [28]:
import unidecode

In [ ]:
# Dictionnaire pour stocker mot-pages

In [29]:
alphabet="0123456789abcdefghijklmnopqrstuvwxyz"

In [32]:
def initialisation(vocab): # initialise le dictionnaire voulu avec une liste vide pour la value et l'indice des mot pour la clé
    dico={}
    for i in range(len(vocab)):
        dico[str(i)]=[]
    return dico

In [87]:
dico_mot_page = initialisation(vocab)
dico_mot_page

{'0': [], '1': [], '2': [], '3': []}

In [36]:
def mot_page(text, vocab): # retourne la liste de mot (en indice) dans un texte (sous condition d'être dans le vocab)
    mot_=[]
    stop=False
    for i in range(len(text)):
        if ((unidecode.unidecode(text[i].lower()) in alphabet) and stop==False):
            debut = i
            stop=True
        if (not (unidecode.unidecode(text[i].lower()) in alphabet) and stop==True):
            fin = i-1
            stop=False
            mot = unidecode.unidecode(text[debut:fin+1].lower())
            for i in range(len(vocab)):
                if vocab[i]==mot:
                    if not i in mot_:
                        mot_+=[i]
                    break
    return mot_

In [46]:
liste_mot_page1 = mot_page(root[1].findall('text')[0].text, vocab)
liste_mot_page1

[1, 3, 0]

In [48]:
def remplissage (dico_mot_page, liste_mot, id_page): # retourne une complétion de dico_mot_page
    for id_mot in liste_mot:
        dico_mot_page[str(id_mot)]+=[id_page]
    return dico_mot_page

In [88]:
for i in range(len(root)):
    liste_mot_page_i = mot_page(root[i].findall('text')[0].text, vocab)
    dico_mot_page = remplissage(dico_mot_page, liste_mot_page_i, i)
dico_mot_page   

{'0': [0, 1, 2, 3, 4], '1': [0, 1, 2, 3, 4], '2': [], '3': [0, 1, 2, 3, 4]}

### Calcule matrice des coeff TF normalisé

In [14]:
import math
import numpy as np

In [18]:
def TF(m,d,vocab): #m et d est un indice
    mot = vocab[m]
    page=root[d]
    for text in page.findall('text'): # une seul itération
        contenu = text.text
        nb = contenu.count(mot)
    if(nb==0):
        return 0.0
    else:
        return 1+math.log10(nb)

In [16]:
def Nd(d, vocab):
    nd=0
    for i in range(len(vocab)):
        nd += TF(i, d, vocab)
    return math.sqrt(nd**2)

In [59]:
longueur_mot = len(vocab)
longueur_page = len(root)
tab_TF_normalise = np.zeros(longueur_mot*longueur_page)
tab_TF_normalise = tab_TF_normalise.reshape((longueur_mot,longueur_page))
for j in range(longueur_page):
    nd = Nd(j, vocab)
    for i in range(longueur_mot):
        tab_TF_normalise[i][j]=TF(i,j,vocab)/nd

In [60]:
print(tab_TF_normalise)

[[0.38641542 0.32733512 0.3322783  0.29070882 0.3535805 ]
 [0.28494952 0.31605783 0.31698595 0.28486327 0.31110605]
 [0.         0.         0.         0.12185722 0.        ]
 [0.32863506 0.35660705 0.35073575 0.30257068 0.33531345]]


In [70]:
tab_TF_normalise

array([[0.38641542, 0.32733512, 0.3322783 , 0.29070882, 0.3535805 ],
       [0.28494952, 0.31605783, 0.31698595, 0.28486327, 0.31110605],
       [0.        , 0.        , 0.        , 0.12185722, 0.        ],
       [0.32863506, 0.35660705, 0.35073575, 0.30257068, 0.33531345]])

### Retrait de certaines pages du dictionnaire si TF-IDF trop faible

In [89]:
seuil=0.3
for id_mot in range(len(dico_mot_page)):
    lst = dico_mot_page[str(id_mot)].copy()
    for id_page in lst:
        if tab_TF_normalise[id_mot][id_page] < seuil: # Ajouter *IDF(id_mot)
            dico_mot_page[str(id_mot)].remove(id_page)

In [90]:
dico_mot_page

{'0': [0, 1, 2, 4], '1': [1, 2, 4], '2': [], '3': [0, 1, 2, 3, 4]}

En fait ce serais plus rapide de construire un dictionnaire mot-page contenant, pour chaque mot, toutes les page existante du corpus. Puis nous supprimerons les page avec un coefficent TF-IDF null.